In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize 
import json

import seaborn as sns
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics
import lightgbm as lgb

In [2]:
data_ip = pd.read_csv("D:\\Gstore\\train.csv",converters={column: json.loads for column in ['device','geoNetwork','totals','trafficSource'] },dtype={'fullVisitorId':'str'})

In [3]:
import json
import pandas as pd
from pandas.io.json import json_normalize
data_test= pd.read_csv("D:\\Gstore\\test.csv",converters={column: json.loads for column in ['device','geoNetwork','totals','trafficSource'] },dtype={'fullVisitorId':'str'})
data_json_test = pd.DataFrame()
data_json_test[['device','geoNetwork','totals','trafficSource']] = data_test[['device','geoNetwork','totals','trafficSource']]
json_cols = ['device','geoNetwork','totals','trafficSource']
update_data_test = pd.DataFrame()
for i in json_cols:
    update_data_test = pd.concat([update_data_test,pd.DataFrame(json_normalize(data_json_test[i]))],axis = 1)
update_data_test = pd.DataFrame(update_data_test)
data_prep_test = update_data_test
data_test = data_test.drop(json_cols,axis=1)
data_json_test = pd.concat([data_test,update_data_test],axis=1)

In [4]:
data_ip.head()

channelGrouping      date  \
0  Organic Search  20160902   
1  Organic Search  20160902   
2  Organic Search  20160902   
3  Organic Search  20160902   
4  Organic Search  20160902   

                                              device        fullVisitorId  \
0  {'browser': 'Chrome', 'browserVersion': 'not a...  1131660440785968503   
1  {'browser': 'Firefox', 'browserVersion': 'not ...   377306020877927890   
2  {'browser': 'Chrome', 'browserVersion': 'not a...  3895546263509774583   
3  {'browser': 'UC Browser', 'browserVersion': 'n...  4763447161404445595   
4  {'browser': 'Chrome', 'browserVersion': 'not a...    27294437909732085   

                                          geoNetwork  \
0  {'continent': 'Asia', 'subContinent': 'Western...   
1  {'continent': 'Oceania', 'subContinent': 'Aust...   
2  {'continent': 'Europe', 'subContinent': 'South...   
3  {'continent': 'Asia', 'subContinent': 'Southea...   
4  {'continent': 'Europe', 'subContinent': 'North...   

                        sessionId  socialEngagementType  \
0  1131660440785968503_1472830385  Not Socially Engaged   
1   377306020877927890_1472880147  Not Socially Engaged   
2  3895546263509774583_1472865386  Not Socially Engaged   
3  4763447161404445595_1472881213  Not Socially Engaged   
4    27294437909732085_1472822600  Not Socially Engaged   

                                              totals  \
0  {'visits': '1', 'hits': '1', 'pageviews': '1',...   
1  {'visits': '1', 'hits': '1', 'pageviews': '1',...   
2  {'visits': '1', 'hits': '1', 'pageviews': '1',...   
3  {'visits': '1', 'hits': '1', 'pageviews': '1',...   
4  {'visits': '1', 'hits': '1', 'pageviews': '1',...   

                                       trafficSource     visitId  visitNumber  \
0  {'campaign': '(not set)', 'source': 'google', ...  1472830385            1   
1  {'campaign': '(not set)', 'source': 'google', ...  1472880147            1   
2  {'campaign': '(not set)', 'source': 'google', ...  1472865386            1   
3  {'campaign': '(not set)', 'source': 'google', ...  1472881213            1   
4  {'campaign': '(not set)', 'source': 'google', ...  1472822600            2   

   visitStartTime  
0      1472830385  
1      1472880147  
2      1472865386  
3      1472881213  
4      1472822600

In [5]:
data_json = pd.DataFrame()
data_json[['device','geoNetwork','totals','trafficSource']] = data_ip[['device','geoNetwork','totals','trafficSource']]


In [6]:
json_cols = ['device','geoNetwork','totals','trafficSource']
update_data = pd.DataFrame()
for i in json_cols:
    update_data = pd.concat([update_data,pd.DataFrame(json_normalize(data_json[i]))],axis = 1)
update_data

browser                    browserSize  \
0                  Chrome  not available in demo dataset   
1                 Firefox  not available in demo dataset   
2                  Chrome  not available in demo dataset   
3              UC Browser  not available in demo dataset   
4                  Chrome  not available in demo dataset   
5                  Chrome  not available in demo dataset   
6                  Chrome  not available in demo dataset   
7                  Chrome  not available in demo dataset   
8       Internet Explorer  not available in demo dataset   
9                 Firefox  not available in demo dataset   
10                 Chrome  not available in demo dataset   
11                 Safari  not available in demo dataset   
12                 Safari  not available in demo dataset   
13                 Safari  not available in demo dataset   
14                 Chrome  not available in demo dataset   
15                 Chrome  not available in demo dataset   
16                 Chrome  not available in demo dataset   
17      Internet Explorer  not available in demo dataset   
18                 Chrome  not available in demo dataset   
19                 Chrome  not available in demo dataset   
20                 Chrome  not available in demo dataset   
21                 Chrome  not available in demo dataset   
22                 Safari  not available in demo dataset   
23                 Chrome  not available in demo dataset   
24                 Chrome  not available in demo dataset   
25      Internet Explorer  not available in demo dataset   
26                Firefox  not available in demo dataset   
27                 Safari  not available in demo dataset   
28                 Chrome  not available in demo dataset   
29                 Chrome  not available in demo dataset   
...                   ...                            ...   
903623             Chrome  not available in demo dataset   
903624             Chrome  not available in demo dataset   
903625            Firefox  not available in demo dataset   
903626             Chrome  not available in demo dataset   
903627             Safari  not available in demo dataset   
903628             Chrome  not available in demo dataset   
903629             Chrome  not available in demo dataset   
903630             Chrome  not available in demo dataset   
903631             Chrome  not available in demo dataset   
903632         Opera Mini  not available in demo dataset   
903633             Chrome  not available in demo dataset   
903634             Chrome  not available in demo dataset   
903635             Chrome  not available in demo dataset   
903636             Chrome  not available in demo dataset   
903637             Chrome  not available in demo dataset   
903638             Chrome  not available in demo dataset   
903639             Chrome  not available in demo dataset   
903640             Safari  not available in demo dataset   
903641             Chrome  not available in demo dataset   
903642  Internet Explorer  not available in demo dataset   
903643             Chrome  not available in demo dataset   
903644             Chrome  not available in demo dataset   
903645    Safari (in-app)  not available in demo dataset   
903646             Chrome  not available in demo dataset   
903647             Chrome  not available in demo dataset   
903648             Chrome  not available in demo dataset   
903649             Chrome  not available in demo dataset   
903650    Android Webview  not available in demo dataset   
903651             Chrome  not available in demo dataset   
903652             Chrome  not available in demo dataset   

                       browserVersion deviceCategory  \
0       not available in demo dataset        desktop   
1       not available in demo dataset        desktop   
2       not available in demo dataset        desktop   
3       not available in demo dataset        desktop   
4       not avail

In [7]:
update_data = pd.DataFrame(update_data)

In [8]:
data_prep = update_data

In [9]:
update_data.shape

(903653, 47)

In [10]:
data_ip = data_ip.drop(json_cols,axis=1)
data_ip.head()

channelGrouping      date        fullVisitorId  \
0  Organic Search  20160902  1131660440785968503   
1  Organic Search  20160902   377306020877927890   
2  Organic Search  20160902  3895546263509774583   
3  Organic Search  20160902  4763447161404445595   
4  Organic Search  20160902    27294437909732085   

                        sessionId  socialEngagementType     visitId  \
0  1131660440785968503_1472830385  Not Socially Engaged  1472830385   
1   377306020877927890_1472880147  Not Socially Engaged  1472880147   
2  3895546263509774583_1472865386  Not Socially Engaged  1472865386   
3  4763447161404445595_1472881213  Not Socially Engaged  1472881213   
4    27294437909732085_1472822600  Not Socially Engaged  1472822600   

   visitNumber  visitStartTime  
0            1      1472830385  
1            1      1472880147  
2            1      1472865386  
3            1      1472881213  
4            2      1472822600

In [11]:
data_json_del = pd.concat([data_ip,update_data],axis=1)

In [12]:
data_json_del.head()

channelGrouping      date        fullVisitorId  \
0  Organic Search  20160902  1131660440785968503   
1  Organic Search  20160902   377306020877927890   
2  Organic Search  20160902  3895546263509774583   
3  Organic Search  20160902  4763447161404445595   
4  Organic Search  20160902    27294437909732085   

                        sessionId  socialEngagementType     visitId  \
0  1131660440785968503_1472830385  Not Socially Engaged  1472830385   
1   377306020877927890_1472880147  Not Socially Engaged  1472880147   
2  3895546263509774583_1472865386  Not Socially Engaged  1472865386   
3  4763447161404445595_1472881213  Not Socially Engaged  1472881213   
4    27294437909732085_1472822600  Not Socially Engaged  1472822600   

   visitNumber  visitStartTime     browser                    browserSize  \
0            1      1472830385      Chrome  not available in demo dataset   
1            1      1472880147     Firefox  not available in demo dataset   
2            1      1472865386      Chrome  not available in demo dataset   
3            1      1472881213  UC Browser  not available in demo dataset   
4            2      1472822600      Chrome  not available in demo dataset   

    ...   adwordsClickInfo.isVideoAd adwordsClickInfo.page  \
0   ...                          NaN                   NaN   
1   ...                          NaN                   NaN   
2   ...                          NaN                   NaN   
3   ...                          NaN                   NaN   
4   ...                          NaN                   NaN   

  adwordsClickInfo.slot   campaign campaignCode isTrueDirect          keyword  \
0                   NaN  (not set)          NaN          NaN   (not provided)   
1                   NaN  (not set)          NaN          NaN   (not provided)   
2                   NaN  (not set)          NaN          NaN   (not provided)   
3                   NaN  (not set)          NaN          NaN  google + online   
4                   NaN  (not set)          NaN         True   (not provided)   

    medium referralPath  source  
0  organic          NaN  google  
1  organic          NaN  google  
2  organic          NaN  google  
3  organic          NaN  google  
4  organic          NaN  google  

[5 rows x 55 columns]

In [13]:
data_json_del['transactionRevenue'] = data_json_del['transactionRevenue'].astype('float')
tru = data_json_del.groupby('fullVisitorId')['transactionRevenue'].sum().reset_index()
plt.scatter(range(tru.shape[0]),np.sort(np.log1p(tru['transactionRevenue'].values)))
plt.show()

From above plot we can see only few users are involved in purchasing i.e; generating revenue to Gstore

In [14]:
cust_rev = (tru['transactionRevenue']>0).sum()
print("number of unique customers having non zero revenue",cust_rev,"also the ratio is ",cust_rev/tru.shape[0])

number of unique customers having non zero revenue 9996 also the ratio is  0.013996726255903731


We can observe that only 1.3% users has non zero revenue

In [15]:
print("number of unique visitors in train set",data_json_del['fullVisitorId'].nunique(),"out of",data_json_del.shape[0])
print("number of unique visitors in test set",data_json_test['fullVisitorId'].nunique(),"out of",data_json_test.shape[0])

number of unique visitors in train set 714167 out of 903653
number of unique visitors in test set 617242 out of 804684


In [16]:
def horizontal_bar_chart(cust_stats, color):
    trace = go.Bar(
        y=cust_stats.index[::-1],
        x=cust_stats.values[::-1],
        showlegend=False,
        orientation = 'h',
        marker=dict(
            color=color,
        ),
    )
    return trace

#Device Browser
cust_stats = data_json_del.groupby('browser')['transactionRevenue'].agg(['size', 'count'])
cust_stats.columns = ["count", "count of non-zero revenue"]
cust_stats = cust_stats.sort_values(by="count", ascending=False)
trace1 = horizontal_bar_chart(cust_stats["count"].head(10), 'rgba(50, 171, 96, 0.6)')
trace2 = horizontal_bar_chart(cust_stats["count of non-zero revenue"].head(10), 'rgba(50, 171, 96, 0.6)')


# Device Category
cust_stats = data_json_del.groupby('deviceCategory')['transactionRevenue'].agg(['size', 'count'])
cust_stats.columns = ["count", "count of non-zero revenue"]
cust_stats = cust_stats.sort_values(by="count", ascending=False)
trace4 = horizontal_bar_chart(cust_stats["count"].head(10), 'rgba(71, 58, 131, 0.8)')
trace5 = horizontal_bar_chart(cust_stats["count of non-zero revenue"].head(10), 'rgba(71, 58, 131, 0.8)')


# Operating system
cust_stats = data_json_del.groupby('operatingSystem')['transactionRevenue'].agg(['size', 'count'])
cust_stats.columns = ["count", "count of non-zero revenue"]
cust_stats = cust_stats.sort_values(by="count", ascending=False)
trace7 = horizontal_bar_chart(cust_stats["count"].head(10), 'rgba(246, 78, 139, 0.6)')
trace8 = horizontal_bar_chart(cust_stats["count of non-zero revenue"].head(10),'rgba(246, 78, 139, 0.6)')


# Creating two subplots
fig = tools.make_subplots(rows=3, cols=2, vertical_spacing=0.04, 
                          subplot_titles=["Device Browser - Count", "Device Browser - Non-zero Revenue Count",
                                          "Device Category - Count",  "Device Category - Non-zero Revenue Count", 
                                          "Device OS - Count", "Device OS - Non-zero Revenue Count"])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace4, 2, 1)
fig.append_trace(trace5, 2, 2)
fig.append_trace(trace7, 3, 1)
fig.append_trace(trace8, 3, 2)

fig['layout'].update(height=1200, width=1200, paper_bgcolor='rgb(233,233,233)', title="Device Plots")
py.iplot(fig, filename='device-plots')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]
[ (3,1) x5,y5 ]  [ (3,2) x6,y6 ]



1. We can see Device browsers are similar in both count and non-zero revenue plots.
2. In Device Category, Mobile is more in count than non zero revenue count.
3. In device OS, Windows count is more in count that non zero revenue.
4. Also Chrome OS count in Non Zero revenue count is more than total count.

In [17]:
# Continent
cust_stats = data_json_del.groupby('continent')['transactionRevenue'].agg(['size', 'count'])
cust_stats.columns = ["count", "count of non-zero revenue"]
cust_stats = cust_stats.sort_values(by="count", ascending=False)
trace1 = horizontal_bar_chart(cust_stats["count"].head(10), 'rgba(50, 171, 96, 0.6)')
trace2 = horizontal_bar_chart(cust_stats["count of non-zero revenue"].head(10), 'rgba(50, 171, 96, 0.6)')

# Sub Continent

cust_stats = data_json_del.groupby('subContinent')['transactionRevenue'].agg(['size', 'count'])
cust_stats.columns = ["count", "count of non-zero revenue"]
cust_stats = cust_stats.sort_values(by="count", ascending=False)
trace4 = horizontal_bar_chart(cust_stats["count"].head(10), 'rgba(71, 58, 131, 0.8)')
trace5 = horizontal_bar_chart(cust_stats["count of non-zero revenue"].head(10), 'rgba(71, 58, 131, 0.8)')


# Network domain
cust_stats = data_json_del.groupby('networkDomain')['transactionRevenue'].agg(['size', 'count'])
cust_stats.columns = ["count", "count of non-zero revenue"]
cust_stats = cust_stats.sort_values(by="count", ascending=False)
trace7 = horizontal_bar_chart(cust_stats["count"].head(10), 'rgba(246, 78, 139, 0.6)')
trace8 = horizontal_bar_chart(cust_stats["count of non-zero revenue"].head(10),'rgba(246, 78, 139, 0.6)')


# Creating two subplots
fig = tools.make_subplots(rows=3, cols=2, vertical_spacing=0.04, 
                          subplot_titles=["Continent - Count", "Continent - Non-zero Revenue Count",
                                          "Sub Continent - Count",  "Sub Continent - Non-zero Revenue Count",
                                          "Network Domain - Count", "Network Domain - Non-zero Revenue Count"])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace4, 2, 1)
fig.append_trace(trace5, 2, 2)
fig.append_trace(trace7, 3, 1)
fig.append_trace(trace8, 3, 2)

fig['layout'].update(height=1200, width=1200, paper_bgcolor='rgb(233,233,233)', title="Geo Plots")
py.iplot(fig, filename='Geo-plots')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]
[ (3,1) x5,y5 ]  [ (3,2) x6,y6 ]



Inferences:

1. We can see America is most popular with more count and in that Northern America is most revenue generated sub continent.
2. We can see unknown.unknown is second most in total count, but it is much varied with Non zero revenue

We can remove all constant features among the train data and also features which are not in test data

In [18]:
const_df = [col for col in data_json_del.columns if data_json_del[col].nunique(dropna=False)==1 ]
const_df

['socialEngagementType',
 'browserSize',
 'browserVersion',
 'flashVersion',
 'language',
 'mobileDeviceBranding',
 'mobileDeviceInfo',
 'mobileDeviceMarketingName',
 'mobileDeviceModel',
 'mobileInputSelector',
 'operatingSystemVersion',
 'screenColors',
 'screenResolution',
 'cityId',
 'latitude',
 'longitude',
 'networkLocation',
 'visits',
 'adwordsClickInfo.criteriaParameters']

In [19]:
print("coulmns in train but not test",set(data_json_del.columns).difference(set(data_json_test.columns)))

coulmns in train but not test {'transactionRevenue', 'campaignCode'}


so we need to remove CampaignCode too

In [20]:
data_json_del = data_json_del.drop(const_df+['campaignCode']+['sessionId'],axis=1)

In [21]:
data_json_test = data_json_test.drop(const_df+['sessionId'],axis=1)

In [22]:
data_json_del["transactionRevenue"].fillna(0, inplace=True)
train_y = data_json_del["transactionRevenue"].values
train_id = data_json_del["fullVisitorId"].values
test_id = data_json_test["fullVisitorId"].values


# label encode the categorical variables and convert the numerical variables to float
cat_cols = ["channelGrouping", "browser", 
            "deviceCategory", "operatingSystem", 
            "city", "continent", 
            "country", "metro",
            "networkDomain", "region", 
            "subContinent", "adContent", 
            "adwordsClickInfo.adNetworkType", 
            "adwordsClickInfo.gclId", 
            "adwordsClickInfo.page", 
            "adwordsClickInfo.slot", "campaign",
            "keyword", "medium", 
            "referralPath", "source",
            'adwordsClickInfo.isVideoAd', 'isTrueDirect']
for col in cat_cols:
    print(col)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(data_json_del[col].values.astype('str')) + list(data_json_test[col].values.astype('str')))
    data_json_del[col] = lbl.transform(list(data_json_del[col].values.astype('str')))
    data_json_test[col] = lbl.transform(list(data_json_test[col].values.astype('str')))






channelGrouping
browser
deviceCategory
operatingSystem
city
continent
country
metro
networkDomain
region
subContinent
adContent
adwordsClickInfo.adNetworkType
adwordsClickInfo.gclId
adwordsClickInfo.page
adwordsClickInfo.slot
campaign
keyword
medium
referralPath
source
adwordsClickInfo.isVideoAd
isTrueDirect


In [23]:
num_cols = ["hits", "pageviews", "visitNumber", "visitStartTime", 'bounces',  'newVisits']    
for col in num_cols:
    data_json_del[col] = data_json_del[col].astype(float)
    data_json_test[col] = data_json_test[col].astype(float)




In [24]:
import datetime
data_json_del['date'] = data_json_del['date'].apply(lambda x: datetime.date(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:])))
dev_set = data_json_del[data_json_del['date']<=datetime.date(2017,5,31)]
val_set = data_json_del[data_json_del['date']>datetime.date(2017,5,31)]
dev_y = np.log1p(dev_set["transactionRevenue"].values)
val_y = np.log1p(val_set["transactionRevenue"].values)

dev_X = dev_set[cat_cols + num_cols] 
val_X = val_set[cat_cols + num_cols] 
test_X = data_json_test[cat_cols + num_cols] 


In [25]:
def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
        "objective" : "regression",
        "metric" : "rmse", 
        "num_leaves" : 30,
        "min_child_samples" : 100,
        "learning_rate" : 0.1,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.5,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
        "verbosity" : -1
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    model = lgb.train(params, lgtrain, 1000, valid_sets=[lgval], early_stopping_rounds=1000, verbose_eval=100)
    
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    pred_val_y = model.predict(val_X, num_iteration=model.best_iteration)
    return pred_test_y, model, pred_val_y

# Training the model #
pred_test, model, pred_val = run_lgb(dev_X, dev_y, val_X, val_y, test_X)

Training until validation scores don't improve for 1000 rounds.
[100]	valid_0's rmse: 1.69586
[200]	valid_0's rmse: 1.69126
[300]	valid_0's rmse: 1.69175
[400]	valid_0's rmse: 1.69449
[500]	valid_0's rmse: 1.69769
[600]	valid_0's rmse: 1.70036
[700]	valid_0's rmse: 1.70235
[800]	valid_0's rmse: 1.70571
[900]	valid_0's rmse: 1.70779
[1000]	valid_0's rmse: 1.70937
Did not meet early stopping. Best iteration is:
[254]	valid_0's rmse: 1.69087


In [27]:
sub_df = pd.DataFrame({"fullVisitorId":test_id})
pred_test[pred_test<0] = 0
sub_df["PredictedLogRevenue"] = np.expm1(pred_test)
sub_df = sub_df.groupby("fullVisitorId")["PredictedLogRevenue"].sum().reset_index()
sub_df.columns = ["fullVisitorId", "PredictedLogRevenue"]
sub_df["PredictedLogRevenue"] = np.log1p(sub_df["PredictedLogRevenue"])
sub_df.to_csv("baseline_lgb.csv", index=False)

In [28]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [29]:
dev_set[cat_cols + num_cols]  = dev_set[cat_cols + num_cols].fillna(0)
dev_X = dev_set[cat_cols + num_cols].values.astype(float)
dev_y = np.log1p(dev_set["transactionRevenue"].values.astype(float))

C:\Users\smadh\Anaconda3\envs\lgb1\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [30]:
from sklearn import tree
clf = tree.DecisionTreeRegressor(max_depth=2)

In [31]:
clf = clf.fit(dev_X,dev_y)

In [32]:
val_set[cat_cols + num_cols] = val_set[cat_cols + num_cols].fillna(0)

C:\Users\smadh\Anaconda3\envs\lgb1\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [33]:
val_X = val_set[cat_cols + num_cols].values.astype(float)

In [34]:
pred_val_tree=clf.predict(val_X)

In [37]:
linreg = LinearRegression()
linreg.fit(dev_X,dev_y)
pred_val_lin = linreg.predict(val_X)
from sklearn import metrics
pred_val_lin[pred_val_lin<0] = 0
val_pred_df_lin = pd.DataFrame({"fullVisitorId":val_set["fullVisitorId"].values})
val_pred_df_lin["transactionRevenue"] = val_set["transactionRevenue"].values
val_pred_df_lin["PredictedRevenue"] = np.expm1(pred_val_lin)
val_pred_df_lin = val_pred_df_lin.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print("RMSE error for Linear Regression model is ",np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df_lin["transactionRevenue"].values), np.log1p(val_pred_df_lin["PredictedRevenue"].values))))

RMSE error for Linear Regression model is  1.9963771910336985


In [35]:
from sklearn import metrics
pred_val_tree[pred_val_tree<0] = 0
val_pred_df_tree = pd.DataFrame({"fullVisitorId":val_set["fullVisitorId"].values})
val_pred_df_tree["transactionRevenue"] = val_set["transactionRevenue"].values
val_pred_df_tree["PredictedRevenue"] = np.expm1(pred_val_tree)
val_pred_df_tree = val_pred_df_tree.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print("RMSE error for Decision Tree with Max_depth 2 is ",np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df_tree["transactionRevenue"].values), np.log1p(val_pred_df_tree["PredictedRevenue"].values))))

RMSE error for Decision Tree with Max_depth 2 is  1.907354481928513


In [36]:
from sklearn import tree
clf = tree.DecisionTreeRegressor(max_depth=6)
clf = clf.fit(dev_X,dev_y)
pred_val_tree=clf.predict(val_X)
from sklearn import metrics
pred_val_tree[pred_val_tree<0] = 0
val_pred_df_tree = pd.DataFrame({"fullVisitorId":val_set["fullVisitorId"].values})
val_pred_df_tree["transactionRevenue"] = val_set["transactionRevenue"].values
val_pred_df_tree["PredictedRevenue"] = np.expm1(pred_val_tree)
val_pred_df_tree = val_pred_df_tree.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print("RMSE error for Decision Tree with Max_Depth 6 is ",np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df_tree["transactionRevenue"].values), np.log1p(val_pred_df_tree["PredictedRevenue"].values)))) 

RMSE error for Decision Tree with Max_Depth 6 is  1.7961302813793891


In [38]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
regr = RandomForestRegressor(max_depth=11,random_state=0,n_estimators=100)
regr.fit(dev_X,dev_y)
pred_val_random = regr.predict(val_X)
pred_val_random[pred_val_random<0] = 0
val_pred_df_random = pd.DataFrame({"fullVisitorId":val_set["fullVisitorId"].values})
val_pred_df_random["transactionRevenue"] = val_set["transactionRevenue"].values
val_pred_df_random["PredictedRevenue"] = np.expm1(pred_val_random)
val_pred_df_random = val_pred_df_random.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print("RMSE error for Random Forest Regressor is ",np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df_random["transactionRevenue"].values), np.log1p(val_pred_df_random["PredictedRevenue"].values))))

RMSE error for Random Forest Regressor is  1.7670536313578376


In [26]:
from sklearn import metrics
pred_val[pred_val<0] = 0
val_pred_df = pd.DataFrame({"fullVisitorId":val_set["fullVisitorId"].values})
val_pred_df["transactionRevenue"] = val_set["transactionRevenue"].values
val_pred_df["PredictedRevenue"] = np.expm1(pred_val)
val_pred_df = val_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print("RMSE error for LightGBM Model is ",np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values))))

RMSE error for LightGBM Model is  1.709925685736863
